In [57]:
SEASON = '2009-10'

In [58]:
# PER GAME CLEANING

import pandas as pd
import myconstants as c

df = pd.read_csv(f'../data/years/{SEASON}/raw/{SEASON}_Player_PerGame.csv')

# Add column [PlayerID]
df['PlayerID'] = df['Player'].str.extract('(' + c.EVERYTHING_AFTER_BACKSLASH + ')')

# [Player] cleaning -- remove backslash + player ID + star (for HOF)
df['Player'] = df['Player'].replace(c.EVERYTHING_AFTER_BACKSLASH, '', regex=True)
df['Player'] = df['Player'].replace('\\\\', '', regex=True)
df['Player'] = df['Player'].replace(c.STAR_AT_END, '', regex=True)

# [Rk] drop -- unnecessary
df = df.drop('Rk', 1)

# [Pos] cleaning -- for dual positions (ex. PG-C) take the first position, discard the second
df['Pos'] = df['Pos'].replace(c.EVERYTHING_AFTER_DASH, '', regex=True)
df['Pos'] = df['Pos'].replace('-', '', regex=True)

# [Age] cleaning
# None

# [Tm] cleaning -- for players that have a TOT (total) row, remove the player's other rows,
# also combine those players' teams on the season and replace TOT with them
df['Tms'] = df.groupby('Player')['Tm'].transform(','.join) # has all teams a player was on
# to remove the duplicate player rows and keep the TOT row, sort in a way that makes the TOT row the last in the df,
# then when dropping duplicate player rows, we can say "keep the last duplicate row in the df", thus keeping the TOT row.
df.loc[df['Tm'] == 'TOT', 'Tm'] = c.ALPHABETICALLY_LAST_STRING
df = df.sort_values('Tm', ascending=True)
df = df.drop_duplicates('Player', keep='last')
# remove TOT and any commas from [Tms]
df['Tms'] = df['Tms'].replace('TOT,', '', regex=True)
df['Tms'] = df['Tms'].replace(',TOT$', '', regex=True)
df = df.drop('Tm', axis=1) # [Tm] no longer needed

# No other columns need to be cleaned

# Move [PlayerID] to the right of [Player] and [Tms] to the right of [PlayerID], stylistic choice
col = df.pop('PlayerID')
df.insert(df.columns.get_loc('Player') + 1, col.name, col)
col = df.pop('Tms')
df.insert(df.columns.get_loc('PlayerID') + 1, col.name, col)

# Sort by player name, stylistic choice
df = df.sort_values('Player')

# write clean data to storage
df.to_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv', index=False)

df

,Player,PlayerID,Tms,Pos,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
435,A.J. Price,priceaj01,IND,PG,23,56,2,15.4,2.6,6.3,0.410,1.1,3.1,0.345,1.5,3.2,0.472,0.494,1.1,1.3,0.800,0.2,1.4,1.6,1.9,0.6,0.1,1.1,0.9,7.3
74,Aaron Brooks,brookaa01,HOU,PG,25,82,82,35.6,7.0,16.2,0.432,2.5,6.4,0.398,4.5,9.8,0.454,0.511,3.0,3.6,0.822,0.7,2.0,2.6,5.3,0.8,0.2,2.8,2.4,19.6
198,Aaron Gray,grayaa01,"CHI,NOH",C,25,32,0,9.7,1.3,2.6,0.512,0.0,0.0,NaN,1.3,2.6,0.512,0.512,0.6,0.9,0.714,1.4,1.9,3.3,0.7,0.3,0.3,0.6,1.7,3.3
326,Acie Law,lawac01,"GSW,CHA,CHI",PG,25,26,1,9.0,1.3,2.9,0.467,0.2,0.6,0.313,1.2,2.3,0.508,0.500,1.5,1.9,0.776,0.1,0.5,0.7,1.0,0.4,0.0,0.7,0.6,4.3
391,Adam Morrison,morriad01,LAL,SF,25,31,0,7.8,1.0,2.7,0.376,0.2,0.7,0.238,0.9,2.1,0.422,0.406,0.2,0.3,0.625,0.3,0.7,1.0,0.6,0.1,0.1,0.3,0.5,2.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,Yakhouba Diawara,diawaya01,MIA,SG,27,6,2,7.3,0.3,1.7,0.200,0.2,1.0,0.167,0.2,0.7,0.250,0.250,0.0,0.0,NaN,0.2,0.5,0.7,0.5,0.2,0.0,0.5,1.3,0.8
299,Yi Jianlian,jianlyi01,NJN,PF,22,52,51,31.8,4.4,10.8,0.403,0.3,0.8,0.366,4.1,10.0,0.406,0.417,3.0,3.7,0.798,2.2,4.9,7.2,0.9,0.7,1.0,1.5,3.3,12.0
447,Zach Randolph,randoza01,MEM,PF,28,81,81,37.7,8.0,16.5,0.488,0.2,0.6,0.288,7.9,15.9,0.496,0.494,4.5,5.7,0.778,4.1,7.7,11.7,1.8,1.0,0.4,2.1,2.8,20.8
420,Zaza Pachulia,pachuza01,ATL,C,25,78,1,14.0,1.5,3.2,0.488,0.0,0.1,0.000,1.5,3.1,0.496,0.488,1.2,1.8,0.650,1.5,2.3,3.8,0.5,0.5,0.4,0.7,2.4,4.3


In [59]:
# ADVANCED PLAYER CLEANING

import pandas as pd
import myconstants as c

df = pd.read_csv(f'../data/years/{SEASON}/raw/{SEASON}_Player_Advanced.csv')

# Add column [PlayerID]
df['PlayerID'] = df['Player'].str.extract('(' + c.EVERYTHING_AFTER_BACKSLASH + ')')

# [Player] cleaning -- remove backslash + player ID + star (for HOF)
df['Player'] = df['Player'].replace(c.EVERYTHING_AFTER_BACKSLASH, '', regex=True)
df['Player'] = df['Player'].replace('\\\\', '', regex=True)
df['Player'] = df['Player'].replace(c.STAR_AT_END, '', regex=True)

# [Rk] drop -- unnecessary
df = df.drop('Rk', 1)

# [Pos] cleaning -- for dual positions (ex. PG-C) take the first position, discard the second
df['Pos'] = df['Pos'].replace(c.EVERYTHING_AFTER_DASH, '', regex=True)
df['Pos'] = df['Pos'].replace('-', '', regex=True)

# [Age] cleaning
# None

# [Tm] cleaning -- for players that have a TOT (total) row, remove the player's other rows,
# also combine those players' teams on the season and replace TOT with them
df['Tms'] = df.groupby('Player')['Tm'].transform(','.join) # has all teams a player was on
# to remove the duplicate player rows and keep the TOT row, sort in a way that makes the TOT row the last in the df,
# then when dropping duplicate player rows, we can say "keep the last duplicate row in the df", thus keeping the TOT row.
df.loc[df['Tm'] == 'TOT', 'Tm'] = c.ALPHABETICALLY_LAST_STRING
df = df.sort_values('Tm', ascending=True)
df = df.drop_duplicates('Player', keep='last')
# remove TOT and any commas from [Tms]
df['Tms'] = df['Tms'].replace('TOT,', '', regex=True)
df['Tms'] = df['Tms'].replace(',TOT$', '', regex=True)
df = df.drop('Tm', axis=1) # [Tm] no longer needed

# remove unnamed columns
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# No other columns need to be cleaned

# Move [PlayerID] to the right of [Player] and [Tms] to the right of [PlayerID], stylistic choice
col = df.pop('PlayerID')
df.insert(df.columns.get_loc('Player') + 1, col.name, col)
col = df.pop('Tms')
df.insert(df.columns.get_loc('PlayerID') + 1, col.name, col)

# Sort by player name, stylistic choice
df = df.sort_values('Player')

# write clean data to storage
df.to_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv', index=False)

df

,Player,PlayerID,Tms,Pos,Age,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
435,A.J. Price,priceaj01,IND,PG,23,56,865,14.0,0.530,0.492,0.212,1.5,9.7,5.6,20.5,2.0,0.2,13.2,22.7,0.4,0.8,1.2,0.065,-0.1,-0.4,-0.5,0.3
74,Aaron Brooks,brookaa01,HOU,PG,25,82,2919,16.0,0.549,0.394,0.224,2.0,6.6,4.2,25.6,1.2,0.3,13.7,25.7,3.9,1.6,5.5,0.091,2.3,-1.6,0.7,1.9
198,Aaron Gray,grayaa01,"CHI,NOH",C,25,32,311,16.6,0.551,0.000,0.341,16.7,23.1,19.8,10.6,1.5,2.7,16.0,16.4,0.5,0.4,0.9,0.142,-1.1,-0.2,-1.3,0.1
326,Acie Law,lawac01,"GSW,CHA,CHI",PG,25,26,234,16.0,0.585,0.213,0.653,1.4,6.6,4.1,17.5,2.3,0.4,15.0,21.7,0.3,0.2,0.6,0.115,-0.6,-0.4,-1.0,0.1
391,Adam Morrison,morriad01,LAL,SF,25,31,241,7.4,0.418,0.247,0.094,4.7,10.1,7.4,11.3,0.6,0.6,10.2,18.3,-0.2,0.2,0.0,0.009,-3.6,-1.1,-4.7,-0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,Yakhouba Diawara,diawaya01,MIA,SG,27,6,44,-3.7,0.250,0.600,0.000,2.7,8.0,5.3,9.7,1.2,0.0,23.1,13.9,-0.2,0.0,-0.1,-0.124,-10.5,-0.9,-11.4,-0.1
299,Yi Jianlian,jianlyi01,NJN,PF,22,52,1655,12.3,0.481,0.073,0.343,7.7,18.7,13.0,4.9,1.2,2.3,10.6,20.1,0.0,1.3,1.3,0.037,-2.8,-1.3,-4.1,-0.9
447,Zach Randolph,randoza01,MEM,PF,28,81,3051,21.2,0.546,0.039,0.348,12.6,23.8,18.2,8.2,1.3,0.8,10.1,24.6,7.0,2.7,9.7,0.153,2.7,-1.4,1.3,2.5
420,Zaza Pachulia,pachuza01,ATL,C,25,78,1089,13.1,0.539,0.016,0.581,12.4,19.0,15.7,5.6,1.8,2.2,15.8,15.5,1.2,1.5,2.7,0.121,-2.7,0.2,-2.5,-0.1


In [60]:
# TEAM STANDINGS CLEANING

import pandas as pd
import myconstants as c

df = pd.read_csv(f'../data/years/{SEASON}/raw/{SEASON}_Team_Standings.csv')

pd.set_option('display.max_columns', None)

# [Rk] drop -- unnecessary
df = df.drop('Rk', 1)

# Conference columns renaming
df = df.rename(columns={'E': 'East','W': 'West'})

# Division columns renaming
df = df.rename(columns={'A': 'Atlantic', 'C': 'Central',
    'SE': 'Southeast', 'NW': 'Northwest',
    'P': 'Pacific', 'SW': 'Southwest'})

# All-Star columns renaming
df = df.rename(columns={'Pre': 'PreAllStar', 'Post': 'PostAllStar'})

# Margin columns renaming
df = df.rename(columns={'≤3': '≤3Margin', '≥10': '≥10Margin'})

# Month columns renaming
# None needed

# Separate W-L columns into 2 columns: W and L
WinLossColumns = df.columns.drop('Team')
for col in WinLossColumns:
    df[['{}W'.format(col), '{}L'.format(col)]] = df[col].str.split('-', n=1, expand=True)
df = df.drop(WinLossColumns, axis=1)

# Sort by team name, stylistic choice
df = df.sort_values('Team')

# write clean data to storage
df.to_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv', index=False)

df

,Team,OverallW,OverallL,HomeW,HomeL,RoadW,RoadL,EastW,EastL,WestW,WestL,AtlanticW,AtlanticL,CentralW,CentralL,SoutheastW,SoutheastL,NorthwestW,NorthwestL,PacificW,PacificL,SouthwestW,SouthwestL,PreAllStarW,PreAllStarL,PostAllStarW,PostAllStarL,≤3MarginW,≤3MarginL,≥10MarginW,≥10MarginL,OctW,OctL,NovW,NovL,DecW,DecL,JanW,JanL,FebW,FebL,MarW,MarL,AprW,AprL
5,Atlanta Hawks,53,29,34,7,19,22,32,20,21,9,13,5,11,7,8,8,7,3,7,3,7,3,33,18,20,11,4,6,31,9,2,0,10,5,9,5,9,6,7,5,11,5,5,3
8,Boston Celtics,50,32,24,17,26,15,33,19,17,13,13,3,10,8,10,8,7,3,5,5,5,5,32,18,18,14,8,6,23,10,3,0,10,4,10,4,6,8,7,5,11,6,3,5
14,Charlotte Bobcats,44,38,31,10,13,28,27,25,17,13,8,10,9,9,10,6,4,6,8,2,5,5,26,25,18,13,9,7,21,11,1,2,6,7,5,9,12,4,4,7,11,6,5,3
16,Chicago Bulls,41,41,24,17,17,24,28,24,13,17,10,8,10,6,8,10,4,6,3,7,6,4,25,26,16,15,7,5,15,21,1,1,5,8,7,8,10,5,8,6,4,11,6,2
0,Cleveland Cavaliers,61,21,35,6,26,15,38,14,23,7,15,3,12,4,11,7,7,3,10,0,6,4,43,11,18,10,7,8,32,5,2,2,10,3,14,3,12,3,8,3,13,2,2,5
3,Dallas Mavericks,55,27,28,13,27,14,22,8,33,19,6,4,9,1,7,3,9,9,14,4,10,6,32,20,23,7,9,2,21,14,2,1,11,4,9,5,8,7,9,4,11,4,5,2
6,Denver Nuggets,53,29,34,7,19,22,19,11,34,18,7,3,7,3,5,5,12,4,11,7,11,7,35,18,18,11,9,4,29,13,2,0,10,5,8,7,12,3,7,5,9,7,5,2
23,Detroit Pistons,27,55,17,24,10,31,18,34,9,21,7,11,2,14,9,9,3,7,2,8,4,6,18,33,9,22,4,1,7,29,1,2,5,9,5,10,4,10,6,7,2,13,4,4
25,Golden State Warriors,26,56,18,23,8,33,11,19,15,37,7,3,3,7,1,9,6,12,5,11,4,14,14,37,12,19,2,8,14,28,0,2,6,8,3,12,4,11,4,8,4,12,5,3
15,Houston Rockets,42,40,23,18,19,22,14,16,28,24,7,3,4,6,3,7,11,7,8,10,9,7,27,24,15,16,5,5,17,15,2,1,7,7,11,5,5,9,4,7,8,8,5,3
